# Small notebook for settings up Hydro maps for Wflow

### Getting the dependencies

In [1]:
import os
import geopandas as gpd
import numpy as np
import xarray as xr

import pyflwdir
from hydromt import DataCatalog, flw

### Loading Data

In [2]:
dc = DataCatalog(data_libs=["../../data/DEM/dem.yml"])
dem = dc.get_rasterdataset("ned")

dem

<xarray.DataArray 'dem_one' (y: 10812, x: 10812)>
dask.array<getitem, shape=(10812, 10812), dtype=float32, chunksize=(6000, 6000), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 -122.0 -122.0 -122.0 ... -121.0 -121.0 -121.0
  * y            (y) float64 48.0 48.0 48.0 48.0 48.0 ... 47.0 47.0 47.0 47.0
    spatial_ref  int32 0
Attributes:
    _FillValue:   -999999.0
    source_file:  dem_one.vrt

### Deriving other DEM and flow directions related data

Once you are satisfied with your flow direction map, you can create additional derived variables like upstream area or streamorder that can prove useful for example to build a model based on ``subbasin`` region.

Here are some examples how to do that using PyFlwdir methods.

In [3]:
merit_adapted = dem.to_dataset(name="elevtn")
merit_adapted["flwdir"] = flw.d8_from_dem(
    da_elv=dem,
    gdf_stream=None,
    max_depth=-1,  # no local pits
    outlets="edge",
    idxs_pit=None,
)

In [29]:
# Set the deminsions
dims = merit_adapted.raster.dims

# Create a PyFlwDir object from the dataset
flwdir = flw.flwdir_from_da(merit_adapted["flwdir"])

# uparea
uparea = flwdir.upstream_area(unit="km2")
merit_adapted["uparea"] = xr.Variable(dims, uparea, attrs=dict(_FillValue=-9999))

2074.711199618479

In [5]:
# Upscale the pyflwdir object
da, flwdir_out = flw.upscale_flwdir(
    merit_adapted,
    flwdir,
    8,
    method="ihu",
    uparea_name = "uparea",
)

In [30]:
# Produce new output
merit_adapted = da.to_dataset().reset_coords(["x_out", "y_out", "idx_out"])
merit_adapted.raster.set_crs(4269)
un_idx = np.unravel_index(merit_adapted["idx_out"].values, dem.shape)
merit_adapted["elevtn"] = xr.Variable(dims, dem.values[un_idx[0], un_idx[1]], attrs=dict(_FillValue=dem.raster.nodata))
# uparea
uparea = flwdir_out.upstream_area(unit="km2")
merit_adapted["uparea"] = xr.Variable(dims, uparea, attrs=dict(_FillValue=-9999))
merit_adapted

<xarray.Dataset>
Dimensions:      (y: 1352, x: 1352)
Coordinates:
  * y            (y) float64 48.0 48.0 48.0 48.0 48.0 ... 47.0 47.0 47.0 47.0
  * x            (x) float64 -122.0 -122.0 -122.0 ... -121.0 -121.0 -121.0
    spatial_ref  int32 0
Data variables:
    x_out        (y, x) float64 -122.0 -122.0 -122.0 ... -121.0 -121.0 -121.0
    y_out        (y, x) float64 48.0 48.0 48.0 48.0 48.0 ... 47.0 47.0 47.0 47.0
    idx_out      (y, x) int32 75686 32444 17 ... 116899324 116899332 116899340
    flwdir       (y, x) uint8 4 16 0 2 2 1 0 1 4 4 ... 16 16 16 16 1 0 16 16 16
    elevtn       (y, x) float32 145.4 145.7 147.3 ... 1.703e+03 1.736e+03
    uparea       (y, x) float64 0.009079 0.00454 0.00454 ... 0.01388 0.004627

In [38]:
# stream order
strord = flwdir_out.stream_order()
merit_adapted["strord"] = xr.Variable(dims, strord)
merit_adapted["strord"].raster.set_nodata(255)

# slope
slope = pyflwdir.dem.slope(
    elevtn=merit_adapted["elevtn"].values,
    nodata=merit_adapted["elevtn"].raster.nodata,
    latlon=False,  # True if geographic crs, False if projected crs
    transform=merit_adapted["elevtn"].raster.transform,
)
merit_adapted["slope"] = xr.Variable(dims, slope)
merit_adapted["slope"].raster.set_nodata(merit_adapted["elevtn"].raster.nodata)

# basin at the pits locations
basins = flwdir_out.basins(idxs=flwdir_out.idxs_pit).astype(np.int32)
merit_adapted["basins"] = xr.Variable(dims, basins, attrs=dict(_FillValue=0))

# basin index file
gdf_basins = merit_adapted["basins"].raster.vectorize()

merit_adapted = merit_adapted.drop_vars(["x_out", "y_out", "idx_out"])
merit_adapted


<xarray.Dataset>
Dimensions:      (y: 1352, x: 1352)
Coordinates:
  * y            (y) float64 48.0 48.0 48.0 48.0 48.0 ... 47.0 47.0 47.0 47.0
  * x            (x) float64 -122.0 -122.0 -122.0 ... -121.0 -121.0 -121.0
    spatial_ref  int32 0
Data variables:
    flwdir       (y, x) uint8 4 16 0 2 2 1 0 1 4 4 ... 16 16 16 16 1 0 16 16 16
    elevtn       (y, x) float32 145.4 145.7 147.3 ... 1.703e+03 1.736e+03
    uparea       (y, x) float64 0.009079 0.00454 0.00454 ... 0.01388 0.004627
    strord       (y, x) uint8 1 1 1 1 1 1 1 2 2 1 1 1 ... 2 2 2 2 2 1 1 2 2 2 1
    slope        (y, x) float32 1.856e+03 2.186e+03 ... 3.72e+04 1.499e+04
    basins       (y, x) int32 167 167 1 168 168 2 ... 1009 1009 1009 1009 1009

### Lets write this stuff

In [39]:
output_path = r"p:\1000365-002-wflow\tmp\usgs_wflow\data\HYDRO"
filename = "ned_hydro_ihu"

# Exporting
# Export the gridded data as tif files in a new folder
# export the hydrography data as tif files (one per variable)
merit_adapted.raster.to_mapstack(
    root=os.path.join(output_path, filename),
    driver="GTiff",
)

# export the basin index as geosjon
gdf_basins.to_file(
    os.path.join(output_path, f"{filename}_basins.geojson"), driver="GeoJSON"
)

: 

### Append Catalog

In [ ]:
add_yml = f"""ned_hydro:
  crs: 4269
  data_type: RasterDataset
  driver: raster
  kwargs:
    chunks:
      x: 6000
      y: 6000
  meta:
    category: topography
    processing_notes: prepared from NED by deriving flow directions using pyflwdir.
    processing_script: create_hydro.py
  path: HYDRO/local_dem/{{variable}}.tif
  rename:
    slope: lndslp

ned_hydro_index:
  crs: 4269
  data_type: GeoDataFrame
  driver: vector
  meta:
    category: topography
    processing_notes: prepared from NED by deriving flow directions using pyflwdir.
    processing_script: create_hydro.py
  path: ned_hydro_basins.geojson
  rename:
    value: basid"""

with open(r"p:\1000365-002-wflow\tmp\usgs_wflow\data\catalog.yml", "a") as w:
    w.write("\n")
    w.write(add_yml)
    w.write("\n")